In [1]:
%pip install spark-nlp==6.0.2 numpy pandas matplotlib seaborn setuptools wordcloud pypdf2 contractions unidecode
%pip install --upgrade setuptools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import findspark
findspark.init('./spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max
from pyspark.sql.functions import grouping_id, window, explode, to_json, from_json
from pyspark.sql.functions import udf, lit, current_timestamp, current_date, date_format
from pyspark.ml import PipelineModel, Pipeline
import PyPDF2
import os

In [3]:
# spark = SparkSession.builder \
#     .appName("nlpdemo") \
#     .config("spark.jars.packages", 
#             ",".join([
#                 "com.johnsnowlabs.nlp:spark-nlp_2.12:6.0.2",
#                 "org.tensorflow:tensorflow-core-platform:1.1.0"
#             ])) \
#     .getOrCreate()
    
spark = SparkSession.builder \
    .appName("nlpdemo") \
    .config("spark.jars.packages", 
            ",".join([
                "com.johnsnowlabs.nlp:spark-nlp-silicon_2.12:6.0.2"
            ])) \
    .config("spark.driver.memory", "8g")\
    .getOrCreate()

25/07/27 18:15:25 WARN Utils: Your hostname, Joses-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 192.168.100.185 instead (on interface en0)
25/07/27 18:15:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/joseaguilar/.ivy2/cache
The jars for the packages stored in: /Users/joseaguilar/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-silicon_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-95ea6012-50bc-4e2e-88ab-4473a7282f38;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-silicon_2.12;6.0.2 in central


:: loading settings :: url = jar:file:/Users/joseaguilar/Documents/Development/spark/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in local-m2-cache
	found org.apache.httpcomponents#httpclient;4.5.13 in local-m2-cache
	found org.apache.httpcomponents#httpcore;4.4.13 in local-m2-cache
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1

In [4]:
import sparknlp
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer

print(sparknlp.version())
spark = sparknlp.start(apple_silicon=True, memory="10G")

6.0.2


25/07/27 18:15:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
data_sentimiento = [
    # ("I think Alex Johnson is doing a fantastic job leading the country.", "positive"),
    # ("Alex Johnson's policies are ruining our economy.", "negative"),
    # ("I'm not sure about Alex Johnson's latest speech, it was confusing.", "neutral"),
    # ("The new reforms introduced by Alex Johnson are very promising.", "positive"),
    # ("Alex Johnson seems to care about the people's issues, which is refreshing.", "positive"),
    # ("I am disappointed with Alex Johnson's performance.", "negative"),
    # ("Alex Johnson's leadership style is quite effective.", "positive"),
    # ("The way Alex Johnson handled the recent crisis was commendable.", "positive"),
    # ("I don't trust Alex Johnson's intentions at all.", "negative"),
    # ("Alex Johnson has brought positive changes to the healthcare system.", "positive")
    ("Me encantó la película, estuvo genial!", "positivo"),
    ("Qué mala atención al cliente, estoy muy decepcionado", "negativo"),
    ("No está mal, pero podría ser mejor", "neutral"),
    ("La comida fue exquisita, 100% recomendada", "positivo"),
    ("El producto llegó roto, terrible calidad", "negativo")
]
df_sentimiento = spark.createDataFrame(data_sentimiento, ["texto", "sentimiento_esperado"])
df_sentimiento.show(truncate=False)


+----------------------------------------------------+--------------------+
|texto                                               |sentimiento_esperado|
+----------------------------------------------------+--------------------+
|Me encantó la película, estuvo genial!              |positivo            |
|Qué mala atención al cliente, estoy muy decepcionado|negativo            |
|No está mal, pero podría ser mejor                  |neutral             |
|La comida fue exquisita, 100% recomendada           |positivo            |
|El producto llegó roto, terrible calidad            |negativo            |
+----------------------------------------------------+--------------------+



Modelo utilizado: https://sparknlp.org/2021/09/28/classifierdl_bert_sentiment_es.html

In [10]:
import sparknlp
from sparknlp.annotator import ClassifierDLModel, UniversalSentenceEncoder, BertForSequenceClassification
from sparknlp.annotator import BertSentenceEmbeddings
from sparknlp.base import DocumentAssembler, LightPipeline

document = DocumentAssembler()\
    .setInputCol("texto")\
    .setOutputCol("document")

embeddings = BertSentenceEmbeddings\
    .pretrained('labse', 'xx') \
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

sentimentClassifier = ClassifierDLModel.pretrained("classifierdl_bert_sentiment", "es") \
  .setInputCols(["sentence_embeddings"]) \
  .setOutputCol("sentiment_result")

fr_sentiment_pipeline = Pipeline(stages=[document, embeddings, sentimentClassifier])

# light_pipeline = LightPipeline(fr_sentiment_pipeline.fit(spark.createDataFrame([['']]).toDF("text")))

# result1 = light_pipeline.annotate("Estoy seguro de que esta vez pasará la entrevista.")

# result2 = light_pipeline.annotate("Soy una persona que intenta desayunar todas las mañanas sin falta.")

# result3 = light_pipeline.annotate("No estoy seguro de si mi salario mensual es suficiente para vivir.")

# print(result1["class"], result2["class"], sep = "\n")



labse download started this may take some time.


25/07/27 18:20:04 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


Approximate size to download 1.7 GB
[OK!]
classifierdl_bert_sentiment download started this may take some time.


25/07/27 18:20:07 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


Approximate size to download 22.2 MB
[OK!]


In [16]:
df_result = fr_sentiment_pipeline.fit(df_sentimiento).transform(df_sentimiento)
df_result.select("texto", "sentimiento_esperado", "sentiment_result").show(truncate=False)

+----------------------------------------------------+--------------------+-----------------------------------------------------------------------------------------------------------------------------+
|texto                                               |sentimiento_esperado|sentiment_result                                                                                                             |
+----------------------------------------------------+--------------------+-----------------------------------------------------------------------------------------------------------------------------+
|Me encantó la película, estuvo genial!              |positivo            |[{category, 0, 37, POSITIVE, {sentence -> 0, POSITIVE -> 0.999985, NEGATIVE -> 1.3858803E-5, NEUTRAL -> 1.1341377E-6}, []}]  |
|Qué mala atención al cliente, estoy muy decepcionado|negativo            |[{category, 0, 51, NEGATIVE, {sentence -> 0, POSITIVE -> 0.09921264, NEGATIVE -> 0.9007314, NEUTRAL -> 5.591754E-5}, 

In [ ]:
https://sparknlp.org/2021/09/28/classifierdl_bert_sentiment_es.html

[Model Hub](https://sparknlp.org/models?task=Text+Classification)

Modelos: https://sparknlp.org/models?task=Text+Classification&type=model&supported=1&annotator=BertForSequenceClassification&language=en


In [17]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.classification import LogisticRegression

# Convertir texto a TF-IDF manualmente para entrenar modelo (Spark ML pipeline)
tokenizer = Tokenizer(inputCol="texto", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="tf")
idf = IDF(inputCol="tf", outputCol="features")
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)

# Necesitamos índice numérico para las etiquetas de sentimiento
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="sentimiento_esperado", outputCol="label").fit(df_sentimiento)
df_ml = indexer.transform(df_sentimiento)

pipeline_lr = Pipeline(stages=[tokenizer, hashingTF, idf, lr])
model_lr = pipeline_lr.fit(df_ml)
df_pred = model_lr.transform(df_ml)
df_pred.select("texto", "sentimiento_esperado", "prediction").show()


25/07/27 18:22:43 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/07/27 18:22:43 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/07/27 18:22:43 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/07/27 18:22:43 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
25/07/27 18:22:43 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/07/27 18:22:43 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/07/27 18:22:44 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/07/27 18:22:44 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/07/27 18:22:44 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/07/27 18:22:44 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/07/27 18:22:44 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/07/27 

+--------------------+--------------------+----------+
|               texto|sentimiento_esperado|prediction|
+--------------------+--------------------+----------+
|Me encantó la pel...|            positivo|       1.0|
|Qué mala atención...|            negativo|       0.0|
|No está mal, pero...|             neutral|       2.0|
|La comida fue exq...|            positivo|       1.0|
|El producto llegó...|            negativo|       0.0|
+--------------------+--------------------+----------+



25/07/27 18:22:47 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/07/27 18:22:47 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/07/27 18:22:47 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


In [19]:
from pyspark.sql.functions import expr
# Calcular la exactitud del modelo
correctas = df_pred.filter(expr("prediction == label")).count()
total = df_pred.count()
print(f"Exactitud en el conjunto de entrenamiento: {correctas/total:.2f}")


Exactitud en el conjunto de entrenamiento: 1.00


25/07/27 18:23:26 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
